<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Code Explainer

In this lecture we are going to build a code explainer that will create a docstring of a given function! 

## Imports

In [1]:
import inspect  # Used to transform python code to a string
import os
from openai import OpenAI

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

In [2]:
client = OpenAI()

## Prompt design
As we want to obtain explanation about what some piece of code does, as well as a list of the possible bugs, we might want to use two separate prompts and later combine the result:

In [3]:
def create_system_prompt():
    return "Create a high quality docstring for the given python function. Do not include the function in your result."
    
def docstring_prompt(code):
    prompt = f"{code}"
    return prompt

In [4]:
def hello(name):
    print(f'Hello {name}')

In [5]:
print(inspect.getsource(hello))

def hello(name):
    print(f'Hello {name}')



In [6]:
print(docstring_prompt(inspect.getsource(hello)))

def hello(name):
    print(f'Hello {name}')



## OpenAI API Call

In [7]:
docstring_prompt(inspect.getsource(hello))

"def hello(name):\n    print(f'Hello {name}')\n"

In [8]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": create_system_prompt()},
    {"role": "user", "content": docstring_prompt(inspect.getsource(hello))},
  ],
  temperature=0,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  max_tokens=64,
)


In [9]:
print(response.choices[0].message.content)

"""
Prints a greeting message with the given name.

Parameters:
- name (str): The name to be included in the greeting message.

Returns:
None

Example:
>>> hello('Alice')
Hello Alice
"""


## Combine docstring and function

In [10]:
def merge_docstring_and_function(original_function, docstring):
    function_string = inspect.getsource(original_function)
    split = function_string.split("\n")
    first_part, second_part = split[0], split[1:]
    docstring = '\t'.join(docstring.splitlines(True))
    merged_function = first_part + "\n" + '    """' + docstring + '    """' + "\n" + "\n".join(second_part)
    return merged_function

In [11]:
merged_code = merge_docstring_and_function(hello, response.choices[0].message.content)

In [12]:
print(merged_code)

def hello(name):
    """"""
	Prints a greeting message with the given name.
	
	Parameters:
	- name (str): The name to be included in the greeting message.
	
	Returns:
	None
	
	Example:
	>>> hello('Alice')
	Hello Alice
	"""    """
    print(f'Hello {name}')



## Get the docstring of a more complicated function

In [13]:
def test_function(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view


In [14]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": create_system_prompt()},
    {"role": "user", "content": docstring_prompt(inspect.getsource(test_function))},
  ],
  temperature=0,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  max_tokens=256,
)


In [15]:
merged_code = merge_docstring_and_function(test_function,  response.choices[0].message.content)
print(merged_code)

def test_function(test, num_questions):
    """This function takes in a test and the number of questions as input and returns a dictionary representing the student's view of the test. 
	
	The function first initializes a dictionary called `student_view` with a single key-value pair, where the key is 1 and the value is an empty string. This represents the student's view of the first question.
	
	Then, the function iterates over each line in the `test` string. If the line does not start with "Correct Answer:", it appends the line to the value of the current question in the `student_view` dictionary.
	
	If the line does start with "Correct Answer:", it checks if the current question number is less than the total number of questions. If it is, it increments the question number by 1 and adds a new key-value pair to the `student_view` dictionary, where the key is the new question number and the value is an empty string. This represents the student's view of the next question.
	
	Finally, the

In [16]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [17]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": create_system_prompt()},
    {"role": "user", "content": docstring_prompt(inspect.getsource(take))},
  ],
  temperature=0,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  max_tokens=256,
)


In [18]:
merged_code = merge_docstring_and_function(take, response.choices[0].message.content)
print(merged_code)

def take(student_view):
    """This function takes a dictionary `student_view` as input, where the keys represent questions and the values represent the views of those questions. It prompts the user to enter their answers for each question and stores them in a new dictionary `answers`. The function then returns the `answers` dictionary.
	
	Parameters:
	- `student_view` (dict): A dictionary where the keys represent questions and the values represent the views of those questions.
	
	Returns:
	- `answers` (dict): A dictionary where the keys represent questions and the values represent the user's answers for those questions.    """
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

